In [1]:
import csv
import json

In [2]:
def process_csv(filename):
    fh = open(filename)
    reader = csv.reader(fh)
    data = list(reader)
    return data

def write_csv(filename, data):
    with open(filename, "w") as FH:
        writer = csv.writer(FH)
        writer.writerows(data) 
        
def read_json(path):
    with open(path, encoding = "utf-8") as f:
        return json.load(f)

In [3]:
# cs220.csv - roster file from CSL AFS. Has information for all lectures

csl_roster = process_csv("cs220.csv")
csl_roster_header = csl_roster[0]
csl_roster = csl_roster[1:]

In [4]:
csl_roster_header

['login',
 'netid',
 'isis_univid',
 'photo_id',
 'isis_emplid',
 'last_name',
 'first_name',
 'middle_name',
 'campus_email',
 'ferpa_flag']

In [5]:
# Read CSL AFS roster and generate netid to campus ID mapping
csl_roster_dict = {} # KEY: netid; VALUE: campus ID

for row in csl_roster:
    netid = row[csl_roster_header.index("netid")].lower()
    campus_id = row[csl_roster_header.index("isis_univid")]
    if netid not in csl_roster_dict:
        csl_roster_dict[netid] = campus_id
    else:
        print("Strange!", row)

In [6]:
location_map = {
    "BASCOM 272": ["L1", "BAS272", 0],
    "Humanities 3650": ["L2", "HUM2650", 0],
    "PSYCHOLOGY 113": ["A1", "PSY113", 0],
    "Humanities 2650": ["L4", "HUM3650", 0],
    "Ingraham B10": ["L3", "INGB10", 0],
    "Comp Sci 1221": ["M1", "CS1221", 0]
}

location_map = {
    "BASCOM 272": ["L1", "L1", 0],
    "Humanities 3650": ["L2", "L2", 0],
    "PSYCHOLOGY 113": ["A1", "A1", 0],
    "Humanities 2650": ["L4", "L4", 0],
    "Ingraham B10": ["L3", "L3", 0],
    "Comp Sci 1221": ["M1", "M1", 0]
}

In [7]:
# Read roster.json

roster_dict = {} # Key: netid; VALUE: exam location details
exam_name = input("Enter the exam name (exam1, exam2, final): ")

exam_roster_list = []
exam_roster_header = ['id', 'orig', 'venue', 'login', 'lname', 'fname', 'lec', 'lab']
exam_roster_list.append(exam_roster_header)

total_students = 0

roster = read_json("../tools/roster.json")
for student_dict in roster:
    if student_dict["enrolled"]:
        #print(student_dict)
        total_students += 1
        
        if "Online".lower() in student_dict[exam_name].lower():
            continue
        if "testing center".lower() in student_dict[exam_name].lower():
            continue
        
        netid = student_dict["net_id"]
        if netid not in csl_roster_dict:
            print("Couldn't find student details:")
            print(student_dict)
            
            student_list = []
            student_list.append("00000")
            for loc in location_map:
                if loc.lower() in student_dict[exam_name].lower():
                    student_list.append(location_map[loc][0])
                    student_list.append(location_map[loc][1])
                    break
            student_list.append(student_dict['net_id'])
            lname, fname = student_dict['name'].split(", ")
            student_list.append(lname)
            student_list.append(fname)
            student_list.append(student_dict['section'])
            student_list.append(student_dict['section'])
            #print(student_list)
            exam_roster_list.append(student_list)
            location_map[loc][2] += 1
            continue
            
        campus_id = csl_roster_dict[netid]
        exam_details = student_dict[exam_name]
        
        student_list = []
        student_list.append(campus_id)
        for loc in location_map:
            if loc.lower() in student_dict[exam_name].lower():
                student_list.append(location_map[loc][0])
                student_list.append(location_map[loc][1])
                break
        student_list.append(student_dict['net_id'])
        lname, fname = student_dict['name'].split(", ")
        student_list.append(lname)
        student_list.append(fname)
        student_list.append(student_dict['section'])
        student_list.append(student_dict['section'])
        #print(student_list)
        exam_roster_list.append(student_list)
        location_map[loc][2] += 1

Enter the exam name (exam1, exam2, final): exam1


In [8]:
with open(exam_name + "_roster.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(exam_roster_list)

In [9]:
len(exam_roster_list)

681